In [1]:
# Import required libraries
import json
import pandas as pd
import datetime

In [2]:
# Read the patient json file and store it into patients
with open('./dataset/build/patients.json') as f:
    patients = json.load(f)
    
# Read the encounters json file and store it into encounters
with open('./dataset/build/encounters.json') as f1:
    encounters = json.load(f1)       

In [3]:
# List to store patient ids
patIds = []

# List to create managing Org ids
managingOrgs = []

# Iterate through each entry
for entries in patients['entry']:
    # Store patient ids
    patIds.append(entries['resource']['id'])
    
    # Store all org IDs
    managingOrg = entries['resource']['managingOrganization']['reference']
    managingOrgs.append(managingOrg.replace('urn:uuid:',''))

In [4]:
# Creating a dataframe to store patient ids and list of managing org ids
df = pd.DataFrame(columns=['managingOrgId', 'patId'])

# Dump patient and managing org ids
df['managingOrgId'] = managingOrgs
df['patId'] = patIds

# Display the dataframe
df

,managingOrgId,patId
0,065fbe84-3551-4ec3-98a3-0d1198c3cb55,00058c64-996b-42de-8692-d9db634ef859
1,065fbe84-3551-4ec3-98a3-0d1198c3cb55,000eff0a-e969-4b00-941f-579519186980
2,065fbe84-3551-4ec3-98a3-0d1198c3cb55,00159b3f-3d73-4164-a06c-8213afa3ef08
3,065fbe84-3551-4ec3-98a3-0d1198c3cb55,001ca35e-9353-47bb-945d-e78eca0f2cd5
4,065fbe84-3551-4ec3-98a3-0d1198c3cb55,00277ad6-0534-4d7a-aec8-c3805c9c6750
...,...,...
99995,ff906c99-cc6d-4ca3-ade7-5a45b878c86e,ffdc058c-7a8e-4ed0-8255-68f8b16f6e44
99996,ff906c99-cc6d-4ca3-ade7-5a45b878c86e,ffe1dbc1-a5b0-4732-8b7e-6a3336957475
99997,ff906c99-cc6d-4ca3-ade7-5a45b878c86e,ffeb4e99-67de-4e22-9538-340728809706
99998,ff906c99-cc6d-4ca3-ade7-5a45b878c86e,fff4a29e-a0ab-4a76-bc7c-ef2020868900


In [5]:
# List to store patient ids
patIds = []

# List to store service provider org ids
orgList = []

# List to store gp list for all patient ids
gpList = []

# List to store gp list for each patient id entry
gpListperPatient = []

# List to store start date time
startDateTime = []

# Iterate through each entry
for entries in encounters['entry']:
    # Store all service provider org id
    orgId = entries['resource']['serviceProvider']['reference']
    orgList.append(orgId.replace('urn:uuid:',''))    
    
    # Store all patient IDs
    patId = entries['resource']['subject']['reference']
    patIds.append(patId.replace('urn:uuid:',''))
        
    # Run through the list of each gp id for a particular patient and store them in a list
    for gpIds in entries['resource']['participant']:
        gpId = gpIds['individual']['reference']
        gpListperPatient.append(gpId.replace('urn:uuid:',''))
    
    # Storing all the gp ids for a particular patient
    gpList.append(gpListperPatient)
    
    # Empty the list to store the gp ids for next patient id
    gpListperPatient = []
    
    # Split the start date time and combine them as per required date time format
    startDate = entries['resource']['period']['start'].split('T')[0]
    startTime = entries['resource']['period']['start'].split('T')[1].split('-')[0]
    startDateTime.append(startDate + ' ' + startTime)          

In [6]:
# Creating a dataframe to store service provider org id, patient id, gp id and startdatetime
df1 = pd.DataFrame(columns=['serviceProviderOrgId', 'patId', 'gpId', 'startDateTime'])

In [7]:
# Dumping above details into the dataframe
df1['serviceProviderOrgId'] = orgList
df1['patId'] = patIds
df1['gpId'] = gpList
df1['startDateTime'] = startDateTime

# Convert the start and end DateTime into datetime format
df1['startDateTime'] = pd.to_datetime(df1['startDateTime'], format='%Y-%m-%d %H:%M:%S')

# Sort the dataframe based on ascending order of start datetime
df1 = df1.sort_values(['startDateTime'], ascending=[True])
df1 = df1.reset_index(drop=True)

# Display the dataframe
df1

,serviceProviderOrgId,patId,gpId,startDateTime
0,fdccc30d-94dd-4fcb-b212-d380f2a8decb,20c317e4-51a7-4832-9d9b-b2ccd1d04fa3,[bb90db2f-f45d-4ee3-831f-ae0d1d9a08dc],1911-01-06 15:38:20
1,fdccc30d-94dd-4fcb-b212-d380f2a8decb,b23b136a-950c-4269-a9f2-0bcda2126b95,"[ab2e50ea-a0cd-46d7-8626-fcbcceb32ade, d396890...",1911-02-26 13:16:16
2,ce0b5da4-5b12-4648-bfe4-ba21bffefdae,1fe96d6d-ef2b-4cf7-9f88-fc6217394ca1,[dcfd2bcb-1717-4ab6-bd1b-a6ad21c8d3e9],1911-10-24 01:15:08
3,81023f28-8910-4c9d-bcd0-d1ba9abb482b,e52bae46-de31-4afd-b2ef-e7749d8112e1,[a5152e28-3a5c-401d-983e-b9af7744f5b8],1911-11-29 23:02:41
4,81023f28-8910-4c9d-bcd0-d1ba9abb482b,e50f2fe8-1432-4fda-91ca-2ceec656be20,[9fa21384-f122-459a-88dd-b42e3011d0fc],1911-12-12 23:02:41
...,...,...,...,...
156696,ff906c99-cc6d-4ca3-ade7-5a45b878c86e,3ab41ed7-0017-4155-9dd4-860e2f9fd5b9,[4cf278d6-f0e6-4764-8000-53ce5cee365a],2019-01-10 09:17:04
156697,ff906c99-cc6d-4ca3-ade7-5a45b878c86e,8ce3f516-6d1e-44a6-b399-ece64d76cad1,[f11bfd89-68fc-4d93-99d9-0e5a7626e546],2019-01-10 13:13:09
156698,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,5d6dc471-c53a-451d-8eda-5630959f6a04,"[b50576df-781f-4c86-8971-5f1c64677b87, 6121ff3...",2019-01-10 13:37:58
156699,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,5af5d775-42b3-4fab-9b37-a08949690ec9,[55054623-65eb-40ec-9675-4f75b3610b73],2019-01-11 00:46:13


In [8]:
# Creating a dataframe to store service provider org id, patient id, gp id and startdatetime
df2 = pd.DataFrame(columns=['serviceProviderOrgId', 'gpId', 'patId', 'startDateTime'])

# Empty lists to store service provider org id, patient id, gp id and startdatetime
orgList = []
gpIdList = []
patIdList = []
startDateTime = []

# Unlist each gp entry and create a new entry
for i in range(0, len(df1)):
    for eachId in df1.iloc[i]['gpId']:
        gpIdList.append(eachId)
        patIdList.append(df1.iloc[i]['patId'])
        orgList.append(df1.iloc[i]['serviceProviderOrgId'])
        startDateTime.append(df1.iloc[i]['startDateTime'])

# Dump service provider org id, patient id, gp id and startdatetime into dataframe        
df2['gpId'] = gpIdList
df2['patId'] = patIdList
df2['serviceProviderOrgId'] = orgList
df2['startDateTime'] = startDateTime

# Display the dataframe
df2

,serviceProviderOrgId,gpId,patId,startDateTime
0,fdccc30d-94dd-4fcb-b212-d380f2a8decb,bb90db2f-f45d-4ee3-831f-ae0d1d9a08dc,20c317e4-51a7-4832-9d9b-b2ccd1d04fa3,1911-01-06 15:38:20
1,fdccc30d-94dd-4fcb-b212-d380f2a8decb,ab2e50ea-a0cd-46d7-8626-fcbcceb32ade,b23b136a-950c-4269-a9f2-0bcda2126b95,1911-02-26 13:16:16
2,fdccc30d-94dd-4fcb-b212-d380f2a8decb,d3968905-cdc1-474a-80c3-2cce24cf3d3d,b23b136a-950c-4269-a9f2-0bcda2126b95,1911-02-26 13:16:16
3,ce0b5da4-5b12-4648-bfe4-ba21bffefdae,dcfd2bcb-1717-4ab6-bd1b-a6ad21c8d3e9,1fe96d6d-ef2b-4cf7-9f88-fc6217394ca1,1911-10-24 01:15:08
4,81023f28-8910-4c9d-bcd0-d1ba9abb482b,a5152e28-3a5c-401d-983e-b9af7744f5b8,e52bae46-de31-4afd-b2ef-e7749d8112e1,1911-11-29 23:02:41
...,...,...,...,...
213128,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,b50576df-781f-4c86-8971-5f1c64677b87,5d6dc471-c53a-451d-8eda-5630959f6a04,2019-01-10 13:37:58
213129,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,6121ff34-0d3d-4539-b339-2428f021d4ee,5d6dc471-c53a-451d-8eda-5630959f6a04,2019-01-10 13:37:58
213130,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,fd92a5e5-0d2f-4150-8521-0fbb8c747d73,5d6dc471-c53a-451d-8eda-5630959f6a04,2019-01-10 13:37:58
213131,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,55054623-65eb-40ec-9675-4f75b3610b73,5af5d775-42b3-4fab-9b37-a08949690ec9,2019-01-11 00:46:13


In [9]:
# Left join df2 and df on patient ids and filter the records with matching service provider org id and managing org id
finalDf = pd.merge(df2, df, how="left", on=["patId"])
finalDf = finalDf[finalDf['serviceProviderOrgId'] == finalDf['managingOrgId']]
finalDf = finalDf[['serviceProviderOrgId', 'gpId', 'patId', 'startDateTime']]
finalDf

,serviceProviderOrgId,gpId,patId,startDateTime
0,fdccc30d-94dd-4fcb-b212-d380f2a8decb,bb90db2f-f45d-4ee3-831f-ae0d1d9a08dc,20c317e4-51a7-4832-9d9b-b2ccd1d04fa3,1911-01-06 15:38:20
1,fdccc30d-94dd-4fcb-b212-d380f2a8decb,ab2e50ea-a0cd-46d7-8626-fcbcceb32ade,b23b136a-950c-4269-a9f2-0bcda2126b95,1911-02-26 13:16:16
2,fdccc30d-94dd-4fcb-b212-d380f2a8decb,d3968905-cdc1-474a-80c3-2cce24cf3d3d,b23b136a-950c-4269-a9f2-0bcda2126b95,1911-02-26 13:16:16
3,ce0b5da4-5b12-4648-bfe4-ba21bffefdae,dcfd2bcb-1717-4ab6-bd1b-a6ad21c8d3e9,1fe96d6d-ef2b-4cf7-9f88-fc6217394ca1,1911-10-24 01:15:08
4,81023f28-8910-4c9d-bcd0-d1ba9abb482b,a5152e28-3a5c-401d-983e-b9af7744f5b8,e52bae46-de31-4afd-b2ef-e7749d8112e1,1911-11-29 23:02:41
...,...,...,...,...
213128,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,b50576df-781f-4c86-8971-5f1c64677b87,5d6dc471-c53a-451d-8eda-5630959f6a04,2019-01-10 13:37:58
213129,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,6121ff34-0d3d-4539-b339-2428f021d4ee,5d6dc471-c53a-451d-8eda-5630959f6a04,2019-01-10 13:37:58
213130,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,fd92a5e5-0d2f-4150-8521-0fbb8c747d73,5d6dc471-c53a-451d-8eda-5630959f6a04,2019-01-10 13:37:58
213131,46ac7ad6-7487-4dd0-baa0-6e2c8cae76a0,55054623-65eb-40ec-9675-4f75b3610b73,5af5d775-42b3-4fab-9b37-a08949690ec9,2019-01-11 00:46:13


In [10]:
# Reading the input file to store N and test cases
with open('./input_files/Q4/P1/input.txt', 'r') as inputFile:
    inputText = inputFile.readlines()
    for i in range(0,len(inputText)):
        inputText[i] = inputText[i].replace('\n', '')

In [11]:
# Take input from the user for the number of entries
N = int(inputText[0])

# Create an empty list to org id
lines = []

# Take org ids from the user for the number of times the user entered above
for i in range(1,len(inputText)):
    lines.append(inputText[i])  

In [12]:
# To store final outcome for each test case
result = []

# Iterate through each user entry
for orgId in lines:
    # Filter based on user entered org id and extract the gp and patient ids
    filteredDf = finalDf[finalDf['serviceProviderOrgId'] == orgId]
    filteredDf = filteredDf[['gpId', 'patId']]
    
    # Group by gp and patient id to get the number of encounters between them
    groupDf = pd.DataFrame(filteredDf.groupby(['gpId', 'patId']).size()).reset_index()
    groupDf = groupDf.rename(columns={0: 'noOfEncounters'})
    # For each gp id, find no. of encounters with each patient in ascending order wrt gp id and descending order wrt no. of encounters
    groupDf = groupDf.sort_values(['gpId', 'noOfEncounters'], ascending = [True, False]).reset_index(drop=True)
    
    # Pick top 3 records for each gp id
    groupDf = groupDf.groupby(['gpId']).head(3).reset_index(drop=True)
    
    # To find if there are any duplicate patient ids within the dataframe 
    duplicatePatDf = groupDf[groupDf.duplicated(['patId'], keep=False)].reset_index(drop=True)
    
    # If there are duplicated pat ids, it is impossible for gp to distribute 3 coupons without repeating the patients
    if (len(duplicatePatDf) == 0 and (len(groupDf) == len(groupDf['gpId'].drop_duplicates())*3)):
        result.append('POSSIBLE')
    else:
        result.append('IMPOSSIBLE')

In [13]:
# Store the resultant output in a txt file
with open('./problemD1Output.txt', 'w') as f:
    for i in range(0,len(result)):
        text = "Test " + str(i+1) + ": " + str(result[i]) + "\n"
        f.write(text)